# Limpieza de dataset del Censo Nacional de Gobiernos Municipales y Delegacionales 2015
## 1. Introduccion

Indicadores que salen de este dataset:


ID |Descripción
---|:----------
P0306|Programas  de modernización catastral 
P0307|Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial 
P1001|Promedio diario de RSU recolectados
P1003|Número de municipios con disponibilidad de servicios relacionados con los RSU
P1006|Número de municipios con aplicación de programas locales orientados a la GIRSU
P1009|Número de municipios con estudios de generación de RSU

## 2. Descarga de datos

In [13]:
descripciones = {
'P0306' : 'Programas de modernización catastral',
'P0307' : 'Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial',
'P1001' : 'Promedio diario de RSU recolectados',
'P1003' : 'Número de municipios con disponibilidad de servicios relacionados con los RSU',
'P1006' : 'Número de municipios con aplicación de programas locales orientados a la GIRSU',
'P1009' : 'Número de municipios con estudios de generación de RSU',
}

In [598]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import zipfile
import csv
import pprint
import re

In [2]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


In [3]:
root = r'http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/'

links = {
    'P0306' : r'm1/Programa_modernizacion_catastral_cngmd2015_csv.zip', # Programas de modernización catastral
    'P0307' : r'm2/Marco_regulatorio_cngmd2015_csv.zip', # Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial
    'P1001' : r'm6/Rec_RSU_cngmd2015_csv.zip', # Promedio diario de RSU recolectados
    'P1006' : r'm6/Prog_gest_int_RSU_cngmd2015_csv.zip', # Número de municipios con aplicación de programas locales orientados a la GIRSU
    'P1009' : r'm6/Est_gen_comp_RSU_cngmd2015_csv.zip', # Número de municipios con estudios de generación de RSU
}

En el caso del parámetro P1003, los datos se extraen desde 3 archivos. Estos archivos son una base de datos para cada servicio relacionado con los RSU, Utilizando nuevamente el archivo que utiliza P1001 y dos adicionales:

In [4]:
P1003links = { # Número de municipios con disponibilidad de servicios relacionados con los RSU
    1 : r'm6/Rec_RSU_cngmd2015_csv.zip',
    2 : r'm6/Trat_RSU_cngmd2015_csv.zip',
    3 : r'm6/Disp_final_RSU_cngmd2015_csv.zip'
}

In [5]:
# Destino local
destino = r'D:\PCCS\00_RawData\01_CSV\cngmd\2015'

In [6]:
# Descarga de zips para parametros que se encuentran en un solo archivo
m_archivos = {}   # Diccionario para guardar memoria de descarga
for parametro, fuente in links.items():
    file = fuente.split('/')[1]
    remote_path = root+fuente
    local_path = destino + r'\{}'.format(file)
    if os.path.isfile(local_path):
        print('Ya existe el archivo: {}'.format(local_path))
        m_archivos[parametro] = local_path
    else:
        print('Descargando {} ... ... ... ... ... '.format(local_path))
        urllib.request.urlretrieve(remote_path, local_path) #
        m_archivos[parametro] = local_path
        print('se descargó {}'.format(local_path))
    

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Programa_modernizacion_catastral_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Marco_regulatorio_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Rec_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Prog_gest_int_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Est_gen_comp_RSU_cngmd2015_csv.zip


In [7]:
# Descarga de zips para parametro P1003
m_archivos2 = {}   # Diccionario para guardar memoria de descarga
for parametro, fuente in P1003links.items():
    file = fuente.split('/')[1]
    remote_path = root+fuente
    local_path = destino + r'\{}'.format(file)
    if os.path.isfile(local_path):
        print('Ya existe el archivo: {}'.format(local_path))
        m_archivos2[parametro] = local_path
    else:
        print('Descargando {} ... ... ... ... ... '.format(local_path))
        urllib.request.urlretrieve(remote_path, local_path) #
        m_archivos2[parametro] = local_path
        print('se descargó {}'.format(local_path))
    

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Rec_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Trat_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Disp_final_RSU_cngmd2015_csv.zip


In [8]:
# Descompresión de archivos de m_parametro
unzipped = {}
for parametro, comprimido in m_archivos.items():
    target = destino + '\\' + parametro
    if os.path.isfile(target):
        print('Ya existe el archivo: {}'.format(target))
        unzipped[parametro] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped[parametro] = target
        

Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P0306 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P0307 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1006 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1009 ... ... ... ... ... 


In [9]:
# Descompresión de archivos de m_parametro2
unzipped2 = {}
for parametro, comprimido in m_archivos2.items():
    target = destino + '\\P1003\\' + str(parametro)
    if os.path.isfile(target):
        print('Ya existe el archivo: {}'.format(target))
        unzipped2[parametro] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped2[parametro] = target
        

Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\1 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\2 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\3 ... ... ... ... ... 


In [1166]:
# Localizacion de archivos de cada parametro
# Cada parametro tiene rutas y estructuras distintas. En este paso localizo manualmente 
# cada tabla y estructura desde los comprimidos. cada valor del diccionario contiene la ruta hacia
# donde se encuentran las tablas.

cd = r'\conjunto_de_datos'

tablas = {
    'P0306' : destino + r'\P0306' + cd,
    'P0307' : destino + r'\P0307\marco_regulatorio_cngmd2015_dbf' + cd,
    'P1001' : destino + r'\P1001\Rec_RSU_cngmd2015_csv' + cd,
    'P1006' : destino + r'\P1006\Prog_gest_int_RSU_cngmd2015_csv' + cd,
    'P1009' : destino + r'\P1009\Est_gen_comp_RSU_cngmd2015_csv' + cd,
}

# Tablas para P1003
destino2 = destino + r'\P1003'
tablasP1003 = {
    '1' : destino2 + r'\1' + r'\Rec_RSU_cngmd2015_csv' + cd,
    '2' : destino2 + r'\2' + r'\Trat_RSU_cngmd2015_csv' + cd,
    '3' : destino2 + r'\3' + r'\Disp_final_RSU_cngmd2015_csv' + cd,
}

## Construccion de datasets estándar
Los datasets para cada parámetro surgen de diferentes preguntas del censo por lo que sus estructuras son muy desemejantes, razon por la cual:
(1) : Cada parámetro tiene que procesarse individualmente.
(2) : Es conveniente extraer de manera individual los metadatos de cada parámetro. Con este propósito, el siguiente script sirve para extraer los metadatos de cada dataset:

In [1267]:
# Script para extraer metadatos:
def getmeta(path, charcoding):   # Path es el contenido en las variables 'tablas' para cada parametro
    cat = r'\catalogos'
    dic = r'\diccionario_de_datos'
    metadict = {}
    metapath = path.replace(cd, cat)
    metafiles = os.listdir(metapath)
    dicdict = {}
    dicpath = path.replace(cd, dic)
    dicfiles = os.listdir(dicpath)
    for file in metafiles:
        variable = file.replace('.csv', '')
        if file.endswith('.csv'):
            csvpath = metapath+'\\'+file
            metadf = pd.DataFrame.from_csv(csvpath, parse_dates=False)
            try:
                metadf.index = metadf.index.map(str.lower)
            except:
                pass
            metadict[variable] = metadf
        else:
            dothis = input('El archivo {} no es csv, que deseas hacer? [DD]etener [CC]ontinuar'.format(file))
            dothis = dothis.lower()
            if dothis == 'dd':
                raise GeneratorExit('Script detenido por el usuario')
            elif dothis == 'cc':
                continue
            else:
                raise KeyError('No entendi la instruccion {}'.format(dothis))
    for file in dicfiles:
        if file.endswith('.csv'):
            filename = file.replace('.csv', '')
            csvpath = dicpath+'\\'+file
            try:
                dicdf = pd.read_csv(csvpath, skiprows=2, usecols=[1, 2], index_col=0, parse_dates=False).dropna()
            except:
                dicdf = pd.read_csv(csvpath, skiprows=2, usecols=[1, 2], index_col=0, parse_dates=False, encoding = charcoding).dropna()
            dicdf.index = dicdf.index.map(str.lower)
            dicdict[filename] = dicdf
    
    return dicdict, metadict

In [1323]:
# Funcion para revisar metadatos
def queryvar(var, tablelen=10, colprint = 125, dictio = p0306dic, metadat = p0306meta):
    pdefault = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', colprint)   # Expande el espacio para imprimir columnas
    print('"{}" :\n{}'.format(var, dictio.loc[var][0].upper()))
    if len(metadat[var]) > tablelen:
        print('{}\nImprimiendo {} de {} registros'.format('-'*40,tablelen, len(metadat[var])))
    print(metadat[var].head(tablelen))
    pd.set_option('display.max_colwidth', pdefault)   # Regresa la variable de impresion de columnas a su default

### **P0306 - Programas de modernización catastral**

Existencia de programas de modernización catastral en los municipios. 

In [1271]:
# Creacion de diccionarios con metadatos para cada variable de P0306:
par = 'P0306'
p0306dic, p0306meta = getmeta(tablas['P0306'], 'mbcs')
print('Se extrajeron metadatos para las siguientes variables de {}:'.format(par))
for key in p0306meta.keys(): print(key)
print('\nDiccionarios disponibles para {}:'.format(par))
for key in p0306dic.keys(): print(key)

Se extrajeron metadatos para las siguientes variables de P0306:
acc_modr
estructu
inst_enc
perio_ac
prog_mod
tip_inst
ubic_geo

Diccionarios disponibles para P0306:
diccionario_de_datos_programa_modernizacion_catastral_cngmd2015_dbf


In [1272]:
# Para P0306, solo existe una tabla de descripciones por lo que se convierte a un dataframe unico para poder indexar
p0306dic = p0306dic['diccionario_de_datos_programa_modernizacion_catastral_cngmd2015_dbf']
p0306dic

,Descripción
Nombre de la columna,
ubic_geo,Clasificación de las entidades federativas y de éstas a sus municipios.
prog_mod,Clasificación de los municipios de acuerdo con el programa de modernización catastral con el que cuenta.
inst_enc,Clasificación de los municipios de acuerdo con la institución encargada de coordinar y/o ejecutar el programa de moderniz...
tip_inst,Clasificación de los municipios de acuerdo con las administraciones públicas municipales por entidad federativa según ins...
acc_modr,Clasificación de los municipios de acuerdo con las acciones del programa de modernización catastral que se han realizado ...
perio_ac,Clasificación de los municipios de acuerdo con el periodo en el que se realizaron las acciones del programa de modernizac...
estructu,Clasificación de los municipios y las delegaciones del Distrito Federal de acuerdo con la existencia del programa de mode...


In [1273]:
list(p0306dic.index)

['ubic_geo',
 'prog_mod',
 'inst_enc',
 'tip_inst',
 'acc_modr',
 'perio_ac',
 'estructu']

In [1324]:
queryvar('acc_modr')

"acc_modr" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL QUE SE HAN REALIZADO EN ELLOS.
                                                                                               desc_acc
acc_modr                                                                                               
1                                                                        Actualizar el padrón catastral
2                                                         Generar o actualizar la cartografía catastral
3                                                              Adquirir un sistema de gestión catastral
4         Adquirir equipamiento e infraestructura (computadoras, servidores, vehículos, comunicaciones)
5                                                                           Adecuar los espacio físicos
6                                                                      Mejorar los procesos catastrales
7                       

In [1274]:
print('** Descripciones de variables **\n'.upper())
for i in p0306dic.index:
    queryvar(i)
    print('\n')

** DESCRIPCIONES DE VARIABLES **

"ubic_geo" :
CLASIFICACIÓN DE LAS ENTIDADES FEDERATIVAS Y DE ÉSTAS A SUS MUNICIPIOS.
----------------------------------------
Imprimiendo 10 de 2542 registros
          ent    desc_ent  mpio                       des_mpio
ubic_geo                                                      
11034      11  Guanajuato    34                 Santa Catarina
11035      11  Guanajuato    35  Santa Cruz de Juventino Rosas
11036      11  Guanajuato    36             Santiago Maravatío
11037      11  Guanajuato    37           Silao de la Victoria
11038      11  Guanajuato    38                    Tarandacuao
11039      11  Guanajuato    39                       Tarimoro
11040      11  Guanajuato    40                  Tierra Blanca
11041      11  Guanajuato    41                      Uriangato
11042      11  Guanajuato    42              Valle de Santiago
11043      11  Guanajuato    43                       Victoria


"prog_mod" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE A

In [1275]:
# Carga de datos
P0306f = tablas['P0306']+'\\'+os.listdir(tablas['P0306'])[0]
df = pd.read_csv(P0306f, dtype={'ubic_geo':'str'})
df = df.rename(columns = {'ubic_geo':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P0306 = df.where((pd.notnull(df)), None)

El archivo está estructurado de manera inconveniente, teniendo un renglón para cada variable. Lo conveniente es que cada renglón contenga toda la información de un solo municipio.

In [1276]:
# subset para pruebas
test = P0306.loc['15045']
test

,prog_mod,inst_enc,tip_inst,acc_modr,perio_ac,estructu
CVE_MUN,,,,,,
15045,1,None,None,None,None,240500
15045,None,1,1,None,None,240501
15045,None,1,2,None,None,240501
15045,None,2,2,None,None,240501
15045,None,None,None,None,2,240503
15045,None,None,None,1,None,240502
15045,None,None,None,2,None,240502
15045,None,None,None,4,None,240502
15045,None,None,None,5,None,240502


Para corregirlo, primero hacemos dataframes separados para cada variable. Afortunadamente, la columna 'Estructu' sirve para agrupar estructuralmente el dataframe

In [1277]:
queryvar('estructu')

"estructu" :
CLASIFICACIÓN DE LOS MUNICIPIOS Y LAS DELEGACIONES DEL DISTRITO FEDERAL DE ACUERDO CON LA EXISTENCIA DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ADEMÁS DE LA INSTITUCIÓN QUE COORDINA Y/O APOYA EL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ASÍ COMO LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL REALIZADAS Y DEL PERIODO EN QUE SE REALIZARON.
                                                                                    descrip
estructu                                                                                   
240500                       El municipio cuenta con un programa de modernización catastral
240501            Institución que coordina y/o apoya el programa de modernización catastral
240502                          Acciones del programa de modernización catastral realizadas
240503    Periodo en que se realizaron las acciones del programa de modernización catastral


In [1278]:
# ¿El municipio cuenta con un programa de modernización catastral?
P0306_00 = P0306[P0306['estructu'] == 240500]['prog_mod'].astype('int')
print(P0306_00.head(10))
print('-'*50)
queryvar('prog_mod')

CVE_MUN
15045     1
15046     2
15047     2
15048     1
15049     2
15050     1
15051     1
15052     1
15053     1
15054    99
Name: prog_mod, dtype: int32
--------------------------------------------------
"prog_mod" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON EL PROGRAMA DE MODERNIZACIÓN CATASTRAL CON EL QUE CUENTA.
            desc_pro
prog_mod            
1                 Si
2                 No
99        No se sabe


In [1279]:
# ¿En que periodo se realizaron las acciones del programa de modernización catastral?
P0306_03 = P0306[P0306['estructu'] == 240503]['perio_ac'].astype('int')
print(P0306_03.head(10))
print('-'*50)
queryvar('perio_ac')

CVE_MUN
01006     1
02001     3
02002     4
02003     1
02004     1
02005     1
03002     2
03003    99
03008     1
03009     5
Name: perio_ac, dtype: int32
--------------------------------------------------
"perio_ac" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON EL PERIODO EN EL QUE SE REALIZARON LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL.
                                                    descrip
perio_ac                                                   
1                         En el transcurso del presente año
2                                       Durante el año 2014
3                                       Durante el año 2013
4                                        Antes del año 2013
5         No se han realizado acciones para atender el tema
99                                               No se sabe


In [1281]:
# ¿Qué acciones se realizaron?
P0306_02 = P0306[P0306['estructu'] == 240502]['acc_modr'].astype('int').groupby('CVE_MUN').apply(list)
print(P0306_02.head(10))
queryvar('acc_modr')

CVE_MUN
01006                   [1, 5]
02001                [2, 3, 4]
02002       [1, 2, 3, 4, 5, 6]
02003             [1, 2, 3, 4]
02004    [1, 2, 3, 4, 5, 6, 7]
02005          [1, 2, 3, 4, 6]
03002       [1, 2, 3, 4, 5, 6]
03003                     [99]
03008       [1, 2, 3, 4, 5, 6]
03009                      [7]
Name: acc_modr, dtype: object
"acc_modr" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL QUE SE HAN REALIZADO EN ELLOS.
                                                                                               desc_acc
acc_modr                                                                                               
1                                                                        Actualizar el padrón catastral
2                                                         Generar o actualizar la cartografía catastral
3                                                              Adquirir un sistema de gestió

In [1282]:
# ¿Cuantas acciones se realizaron?
P0306_02b = P0306_02.apply(len).rename('n_acc_modr')
P0306_02b.head(10)

CVE_MUN
01006    2
02001    3
02002    6
02003    4
02004    7
02005    5
03002    6
03003    1
03008    6
03009    1
Name: n_acc_modr, dtype: int64

In [1283]:
queryvar('inst_enc')

"inst_enc" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LA INSTITUCIÓN ENCARGADA DE COORDINAR Y/O EJECUTAR EL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ASÍ COMO DE LA QUE SE RECIBEN APOYOS.
                                                                                        desc_ins
inst_enc                                                                                        
1         Institución encargada de coordinar y/o ejecutar el programa de modernización catastral
2                                                        institución de la que se reciben apoyos


In [1284]:
# ¿Que instituciones se han involucrado en la modernizacion catastral, y de qué manera?
P0306_01t = P0306[P0306['estructu'] == 240501][['inst_enc', 'tip_inst']]    # tipo de apoyo e institucion 
P0306_01t.head()

,inst_enc,tip_inst
CVE_MUN,,
01006,1,4
01006,2,4
02001,1,3
02001,2,3
02002,1,3


Se reemplazarán numeros por descripciones en **tip_inst**:

In [1285]:
queryvar('tip_inst')

"tip_inst" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LAS ADMINISTRACIONES PÚBLICAS MUNICIPALES POR ENTIDAD FEDERATIVA SEGÚN INSTITUCIÓN QUE COORDINA EL PROGRAMA DE MODERNIZACIÓN CATASTRAL Y/O RECIBEN EL APOYO.
                                                                  descrip
tip_inst                                                                 
1                         Administración pública de la entidad federativa
2                 Banco Nacional de Obras y Servicios Públicos (BANOBRAS)
3         Secretaría de Desarrollo Agrario, Territorial y Urbano (SEDATU)
4                                                                    Otra


In [1286]:
# Institucion involucrada
instit = {
    1:'Administración pública de la entidad federativa',
    2:'BANOBRAS',
    3:'SEDATU',
    4:'OTRA INSTITUCION'
}

In [1287]:
P0306_01t['tip_inst'] = P0306_01t['tip_inst'].replace(instit)
P0306_01t.head()

,inst_enc,tip_inst
CVE_MUN,,
01006,1,OTRA INSTITUCION
01006,2,OTRA INSTITUCION
02001,1,SEDATU
02001,2,SEDATU
02002,1,SEDATU


Y se separará la columna 'inst_enc' en 2:

In [1288]:
queryvar('inst_enc')

"inst_enc" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LA INSTITUCIÓN ENCARGADA DE COORDINAR Y/O EJECUTAR EL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ASÍ COMO DE LA QUE SE RECIBEN APOYOS.
                                                                                        desc_ins
inst_enc                                                                                        
1         Institución encargada de coordinar y/o ejecutar el programa de modernización catastral
2                                                        institución de la que se reciben apoyos


In [1290]:
P0306_01t1 = P0306_01t[P0306_01t['inst_enc'] == 1]['tip_inst'].groupby('CVE_MUN').apply(list).rename('i_coord_ejecuta')
P0306_01t2 = P0306_01t[P0306_01t['inst_enc'] == 2]['tip_inst'].groupby('CVE_MUN').apply(list).rename('i_otorga_apoyos')

In [1291]:
P0306_01t1.head()

CVE_MUN
01006                                   [OTRA INSTITUCION]
02001                                             [SEDATU]
02002                                             [SEDATU]
02003    [Administración pública de la entidad federativa]
02004                                   [OTRA INSTITUCION]
Name: i_coord_ejecuta, dtype: object

In [1292]:
P0306_01t2.head()

CVE_MUN
01006    [OTRA INSTITUCION]
02001              [SEDATU]
02002              [SEDATU]
02003              [SEDATU]
02004    [OTRA INSTITUCION]
Name: i_otorga_apoyos, dtype: object

Finalmente, se unirán todas las series en un solo dataframe

In [1293]:
# Convertir series en Dataframes
P0306_00 = P0306_00.to_frame()
P0306_03 = P0306_03.to_frame()
P0306_02 = P0306_02.to_frame()
P0306_02b = P0306_02b.to_frame()
P0306_01t1 = P0306_01t1.to_frame()
P0306_01t2 = P0306_01t2.to_frame()

In [1294]:
# Unir dataframes
P0306 = P0306_00.join(P0306_03).join(P0306_02).join(P0306_02b).join(P0306_01t1).join(P0306_01t2)
P0306 = P0306.where((pd.notnull(P0306)), None)

In [1295]:
P0306.head()

,prog_mod,perio_ac,acc_modr,n_acc_modr,i_coord_ejecuta,i_otorga_apoyos
CVE_MUN,,,,,,
15045,1,2,"[1, 2, 4, 5, 6]",5,"[Administración pública de la entidad federativa, BANOBRAS]",[BANOBRAS]
15046,2,None,None,None,None,None
15047,2,None,None,None,None,None
15048,1,2,"[1, 2, 3, 4, 6]",5,[SEDATU],[Administración pública de la entidad federativa]
15049,2,None,None,None,None,None


__Metadatos para P0306__

In [1296]:
P0306meta = {
    'Nombre del Dataset' : 'Censo Nacional de Gobiernos Municipales y Delegacionales 2015',
    'Descripcion del dataset' : 'Censo Nacional de Gobiernos Municipales y Delegacionales 2015',
    'Disponibilidad Temporal' : '2015',
    'Periodo de actualizacion' : 'Bienal',
    'Nivel de Desagregacion' : 'Municipal',
    'Notas' : 's/n',
    'Fuente' : 'INEGI',
    'URL_Fuente' : 'http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/',
    'Dataset base' : '"P0306.xlsx" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015',
}

P0306meta = pd.DataFrame.from_dict(P0306meta, orient='index', dtype=None)
P0306meta.columns = ['Descripcion']
P0306meta = P0306meta.rename_axis('Metadato')
P0306meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,s/n
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P0306.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1297]:
list(P0306meta)

['Descripcion']

In [1298]:
P0306.head()

,prog_mod,perio_ac,acc_modr,n_acc_modr,i_coord_ejecuta,i_otorga_apoyos
CVE_MUN,,,,,,
15045,1,2,"[1, 2, 4, 5, 6]",5,"[Administración pública de la entidad federativa, BANOBRAS]",[BANOBRAS]
15046,2,None,None,None,None,None
15047,2,None,None,None,None,None
15048,1,2,"[1, 2, 3, 4, 6]",5,[SEDATU],[Administración pública de la entidad federativa]
15049,2,None,None,None,None,None


EXPORTAR A EXCEL

In [1300]:
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD\P0306.xlsx'
writer = pd.ExcelWriter(file)
P0306.to_excel(writer, sheet_name = 'P0306')
P0306meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()

### P0307: Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial

Es necesario cambiar el encoding para leer los archivos de este parametro

In [1306]:
# Redefinición de la función para revisar metadatos, porque los datos de la carpeta 'catálogos' de P0307 
# no coinciden con los titulos de las columnas en la carpeta 'Conjunto de datos'.

def getmetab(csvpath, textcoding):
    # Importa el csv
    try: dicdf = pd.read_csv(csvpath, 
                    index_col=0, 
                    parse_dates=False
                   )
    except: dicdf = pd.read_csv(csvpath, 
                    index_col=0, 
                    parse_dates=False, 
                    encoding = textcoding,
                   )
    
    # Renombra las columnas
    dicdf.columns = list(dicdf.iloc[1])
    
    # Crea columna con el indice
    dicdf['text_arc'] = dicdf.index

    # Extrae el nombre del csv fuente en una columna independiente
    def getarc(x):
        try: return re.search('(?<=(o: ))([A-Z])\w+', x).group()
        except: return None
    dicdf['arc'] = dicdf['text_arc'].apply(lambda x: getarc(x))

    # Extrae la descripcion del archivo en una columna independiente
    def getdescarc(x):
        try: return re.search('\(([^)]+)\)', x).group(1)
        except: return None
    dicdf['desc_arc'] = dicdf['text_arc'].apply(lambda x: getdescarc(x))
    
    # Marca columnas que se van a eliminar (Las columnas de donde se sacaron las variables 'arc' y 'desc_arc')
    dicdf['delete1'] = dicdf[list(dicdf.columns)[1:6]].notnull().sum(axis = 1)
    
    # Rellenar valores NaN
    dicdf = dicdf.fillna(method='ffill')
    
    # Eliminar valores marcados previaente
    dicdf = dicdf[dicdf.delete1 != 0]
    
    # Eliminar encabezados de columna repetidos
    dicdf = dicdf[dicdf.Descripción != 'Descripción']
    
    # Asignar nuevo indice y eliminar columna 'arc'
    dicdf = dicdf.set_index('arc')
    
    # Elimina columna delete1
    del dicdf['delete1']
    
    # Renombra la columna de descripciones de codigos
    dicdf.columns.values[5] = 'Descripcion codigos'
    
    # Dame el DataFrame
    return dicdf

In [1307]:
# También es necesario redefinir la función para hacer consultas a los metadatos
def queryvar(filename, var = '', tablelen=10, colprint = 125, dictio = metadatos):
    pdefault = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', colprint)   # Expande el espacio para imprimir columnas
    frame = dictio.loc[filename]
    print('Archivo "{}.csv" {}'.format(filename, '-'*30))   # Muestra el nombre del archivo
    print(frame.iloc[0]['desc_arc'])                     # Muestra la descripcion del archivo
    if var == '': pass
    else:
        print('\n{}{}'.format(var.upper(), '-'*30))                  # Muestra el nombre de la variable
        varframe = frame[frame['Nombre de la \ncolumna'] == var.upper()]   # Haz un subset con los datos de la variable
        varframe = varframe.set_index('Códigos válidos en la columna')
        print(varframe['Descripción'][0])                    # Muestra la descripcion de la variable
        print(varframe[['Descripcion codigos']])             # Imprime las descripciones de codigos

In [1308]:
csvpath = r'D:\PCCS\00_RawData\01_CSV\cngmd\2015\P0307\marco_regulatorio_cngmd2015_dbf\diccionario_de_datos\diccionario_de_datos_marco_regulatorio_cngmd2015.csv'
metadatos = getmetab(csvpath, 'mbcs')

In [1309]:
# Definición de rutas de archivos
par = 'P0307'
P0307files = {}
for file in os.listdir(tablas[par]):
    P0307files[file.replace('.csv', '')] = tablas[par]+'\\'+file

El contenido de los archivos en la carpeta "Conjunto de datos" es el siguiente:

In [1310]:
for file in P0307files.keys():
    print(file)
    queryvar(file.upper())
    print('\n')

asoc_int
Archivo "ASOC_INT.csv" ------------------------------
Contiene variables que caracterizan a los municipios y las delegaciones del Distrito Federal de acuerdo con el tipo de servicio público y/o función objeto de la asociación intergubernamental, el tipo de instrumento regulatorio establecido para el cumplimiento de la prestación de los servicios públicos y el tipo de gobierno con el que se estableció dicha asociación, durante los años 2013 y 2014.


aso_incf
Archivo "ASO_INCF.csv" ------------------------------
Contiene variables que caracterizan a los municipios y las delegaciones del Distrito Federal de acuerdo con la asociación intergubernamental en materia de cultura física y deporte, durante los años 2013 y 2014.


aso_inma
Archivo "ASO_INMA.csv" ------------------------------
Contiene variables que caracterizan a los municipios y las delegaciones del Distrito Federal de acuerdo con la asociación intergubernamental en materia ambiental, durante los años 2013 y 2014.


aso

La información para el parámetro P0307 se encuentra en el archivo M_REGULA.csv

In [1311]:
print('P0307 - {}\n'.format(descripciones['P0307']))
queryvar('m_regula'.upper())

P0307 - Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial

Archivo "M_REGULA.csv" ------------------------------
Contiene variables que caracterizan a los municipios y las delegaciones del Distrito Federal de acuerdo con la cantidad de disposiciones normativas internas por tema que deben ser observadas por las Administraciones Públicas Municipales o Delegacionales, al cierre del año 2014.


In [1312]:
# Carga de datos
P0307f = tablas['P0307']+'\\'+ os.listdir(tablas['P0307'])[4]
df = pd.read_csv(P0307f, dtype={'ubic_geo':'str'})
df = df.rename(columns = {'ubic_geo':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P0307 = df.where((pd.notnull(df)), None)

C:\Users\carlos.arana\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1313]:
P0307.head()

,dis_norm,exi_norm,tema_nis,instrum,tema_nia,estructu,nom_disn,fech_pub,fech_act,tt_dispo,est_ttdi
CVE_MUN,,,,,,,,,,,
08044,10,0,None,None,None,290100,None,16/12/2009,16/12/2009,None,None
08044,11,1,None,None,None,290100,None,NO APLICA,NO APLICA,None,None
08044,12,1,None,None,None,290100,None,NO APLICA,NO APLICA,None,None
08044,13,0,None,None,None,290100,None,20/09/2002,28/04/2010,None,None
08044,14,1,None,None,None,290100,None,NO APLICA,NO APLICA,None,None


In [1314]:
P0307.columns

Index(['dis_norm', 'exi_norm', 'tema_nis', 'instrum', 'tema_nia', 'estructu',
       'nom_disn', 'fech_pub', 'fech_act', 'tt_dispo', 'est_ttdi'],
      dtype='object')

¿Dónde están los datos sobre desarrollo urbano y ordenamiento territorial?

In [1315]:
queryvar('m_regula'.upper(), 'tema_nis')

Archivo "M_REGULA.csv" ------------------------------
Contiene variables que caracterizan a los municipios y las delegaciones del Distrito Federal de acuerdo con la cantidad de disposiciones normativas internas por tema que deben ser observadas por las Administraciones Públicas Municipales o Delegacionales, al cierre del año 2014.

TEMA_NIS------------------------------
Clasificación de los municipios y las delegaciones del Distrito Federal de acuerdo a los temas manejados en las disposiciones normativas internas sustantivas vigentes en la  Administraciones Públicas Municipales o Delegacionales, al cierre del año 2014
                                                                            Descripcion codigos
Códigos válidos en la columna                                                                  
1                                                          Actividades económicas o comerciales
2                                               Administración Pública Municipal o Del

Los datos de DU y OT estan en la columna TEMA_NIS. El código 41 en esta column indica DU y OT

In [1317]:
P0307 = P0307[P0307['tema_nis'] == 41]
P0307.head()

,dis_norm,exi_norm,tema_nis,instrum,tema_nia,estructu,nom_disn,fech_pub,fech_act,tt_dispo,est_ttdi
CVE_MUN,,,,,,,,,,,
20207,None,None,41,0,None,290200,None,None,None,None,0
20285,None,None,41,0,None,290200,None,None,None,None,0
03001,None,None,41,0,None,290200,None,None,None,None,0
21102,None,None,41,0,None,290200,None,None,None,None,0
22006,None,None,41,0,None,290200,None,None,None,None,0


In [1318]:
# Quita las columnas que estén vacías
P0307 = P0307.dropna(axis=1, how = 'all')
P0307.head()

,tema_nis,instrum,estructu,tt_dispo,est_ttdi
CVE_MUN,,,,,
20207,41,0,290200,None,0
20285,41,0,290200,None,0
03001,41,0,290200,None,0
21102,41,0,290200,None,0
22006,41,0,290200,None,0


In [1319]:
# Metadatos
meta = P0306meta
meta.at['Dataset base','Descripcion'] = meta.at['Dataset base','Descripcion'].replace('P0306', 'P0307')
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,s/n
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P0307.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


Exportar archivo

In [1320]:
par = 'P0307'
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD'+'\\'+par+'.xlsx'
writer = pd.ExcelWriter(file)
P0307.to_excel(writer, sheet_name = par)
meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()

### P1001 - Promedio diario de RSU recolectados

In [1201]:
# Rutas de archivos
param = 'P1001'
rutadatos = tablas[param]
rutameta = tablas[param].replace('conjunto_de_datos', 'diccionario_de_datos')
rutameta = rutameta + '\\' + os.listdir(rutameta)[0]
print('{}\n{}'.format(rutadatos, rutameta))

D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001\Rec_RSU_cngmd2015_csv\conjunto_de_datos
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001\Rec_RSU_cngmd2015_csv\diccionario_de_datos\FD_Modulo_6_RSU_2015_secc_i.csv


In [1202]:
# Obtencion de metadatos
# Cada hoja de metadatos es muy muy similar, pero con muy ligeras variaciones
# La unica parte del proceso que es seguro automatizar es la importación del archivo hacia Python
def getmeta(csvpath, textcoding):
    # Importa el csv
    try: 
        dicdf = pd.read_csv(csvpath, 
                    index_col=0, 
                    parse_dates=False
                   )
    except: 
        dicdf = pd.read_csv(csvpath, 
                    index_col=0, 
                    parse_dates=False, 
                    encoding = textcoding,
                   )
    
    # Renombra las columnas
    dicdf.columns = list(dicdf.iloc[1])
    
    # Dame el archivo
    return dicdf

In [1203]:
os.listdir(r'D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001\Rec_RSU_cngmd2015_csv\diccionario_de_datos')

['FD_Modulo_6_RSU_2015_secc_i.csv',
 'FD_Modulo_6_RSU_2015_secc_i_Página_1.png',
 'FD_Modulo_6_RSU_2015_secc_i_Página_2.png']

In [1204]:
metadatos = getmeta(rutameta, 'mbcs')
# Crea columna con el indice
metadatos['Nombre de la \ncolumna'] = metadatos.index

# Extrae el nombre del csv fuente en una columna independiente
def getarc(x):
    try: return x.split(' ')[1]
    except: return None
metadatos['archivo'] = metadatos['Nombre de la \ncolumna'].apply(getarc)

# Extrae la descripcion del archivo en una columna independiente
def getdescarc(x):
    try: return x.split('(')[1].replace(')','')
    except: return None
metadatos['desc_arc'] = metadatos['Nombre de la \ncolumna'].apply(getdescarc)

# En la columna 'arc', reemplaza las celdas cuyo valor es 'de'
metadatos['archivo'] = metadatos['archivo'].replace({'de':None})

# Marca columnas que se van a eliminar (Las columnas de donde se sacaron las variables 'arc' y 'desc_arc')
metadatos['delete1'] = metadatos[list(metadatos.columns)[1:6]].notnull().sum(axis = 1)

# Rellenar valores NaN
metadatos = metadatos.fillna(method='ffill')

# Eliminar valores marcados previaente
metadatos = metadatos[metadatos.delete1 != 0]

# Eliminar columnas sin datos
metadatos = metadatos.dropna(axis = 1, how = 'all')

# Eliminar encabezados de columna repetidos
metadatos = metadatos[metadatos.Descripción != 'Descripción']

# Asignar nuevo indice y eliminar columna 'text_arc'
metadatos = metadatos.set_index('archivo')

# Elimina columna delete1
del metadatos['delete1']

# Renombra la columna de descripciones de codigos
metadatos.columns.values[3] = 'Descripcion codigos'

# Reordena las columnas
neworder = ['Nombre de la \ncolumna', 'Descripción', 'Tipo de dato', 'Rango válido', 'Descripcion codigos', 
            'Pregunta textual', 'Página de Cuestionario', 'Definición', 'desc_arc']

metadatos = metadatos.reindex(columns= neworder)

# Renombra las columnas para que funcionen con queryvar
metadatos = metadatos.rename({'Rango válido':'Códigos válidos en la columna'})

metadatos.head(3)

,Nombre de la columna,Descripción,Tipo de dato,Rango válido,Descripcion codigos,Pregunta textual,Página de Cuestionario,Definición,desc_arc
archivo,,,,,,,,,
secc_i_tr_cngmd15_m6,FOLIO,Identificador de cada cuestionario compuesto por la clave de entidad y por la clave de municipio o delegación.,Carácter,5,"[01001-32058, Not Nul]",FOLIO,1,Identificador de los cuestionarios correspondientes a los municipios y delegaciones del Distrito Federal con base en el C...,"Contiene variables que caracterizan a los municipios de acuerdo a la recolección de residuos sólidos urbanos, durante el ..."
secc_i_tr_cngmd15_m6,ENTIDAD,Clave entidad federativa.,Carácter,2,"[1-32, Not Null]",ENTIDAD,1,Identifica la clave de la entidad donde se ubica el sitio de disposición final con base en el Catálogo Único de Claves de...,"Contiene variables que caracterizan a los municipios de acuerdo a la recolección de residuos sólidos urbanos, durante el ..."
secc_i_tr_cngmd15_m6,MUNICIPIO,Clave municipio o delegación.,Carácter,3,"[1-999, Not Null]",MUNICIPIO,1,"Indica la clave perteneciente al municipio y en el caso del Distrito Federal a una delegación política , cuya base son su...","Contiene variables que caracterizan a los municipios de acuerdo a la recolección de residuos sólidos urbanos, durante el ..."


¿Donde estan los datos?

In [1205]:
metadatos.loc['secc_i_tr_cngmd15_m6'][metadatos.loc['secc_i_tr_cngmd15_m6']['Nombre de la \ncolumna'] == 'P2_2']

,Nombre de la columna,Descripción,Tipo de dato,Rango válido,Descripcion codigos,Pregunta textual,Página de Cuestionario,Definición,desc_arc
archivo,,,,,,,,,
secc_i_tr_cngmd15_m6,P2_2,Cantidad de residuos sólidos recolectada en kilogramos.,Carácter,7,"[1…9999999, null]",Registre la cantidad promedio diaria de residuos sólidos urbanos recolectada en este Municipio o Delegación.,5,"Identifica cúantos kilogramos de residuos sólidos urbanos fueron recolectadas en el municipio o delegación en promedio, c...","Contiene variables que caracterizan a los municipios de acuerdo a la recolección de residuos sólidos urbanos, durante el ..."


Los datos se encuentran en el archivo secc_i_tr_cngmd15_m6, en la columna P2_2

In [1206]:
# Definición de rutas a archivos de datos
Paramfiles = {}
for file in os.listdir(rutadatos):
    Paramfiles[file.replace('.csv', '')] = rutadatos+'\\'+file

for file, path in Paramfiles.items():
    print('{}:\n{}\n'.format(file, path))

secc_i_tr_cngmd15_m6:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001\Rec_RSU_cngmd2015_csv\conjunto_de_datos\secc_i_tr_cngmd15_m6.csv

secc_i_tr_cngmd15_m6_p6_3_2:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001\Rec_RSU_cngmd2015_csv\conjunto_de_datos\secc_i_tr_cngmd15_m6_p6_3_2.csv



In [1209]:
# Carga de datos
P1001f = tablas[param]+'\\'+ os.listdir(tablas[param])[0]
df = pd.read_csv(P1001f, dtype={'folio':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'folio':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1001 = df.where((pd.notnull(df)), None)

In [1208]:
P1001.head(1)

,entidad,municipio,p1,p2_2,p3,p3_1,p6_1,p6_3_1_1_1,p6_3_1_2_1,p6_3_1_3_1,...,p9_3_1_2,p9_3_2_2,p9_3_3_2,p9_3_4_2,p9_3_5_2,p9_3_6_2,p9_3_7_2,p9_3_8_2,p9_3_9_2,p9_3_10_2
CVE_MUN,,,,,,,,,,,,,,,,,,,,,
01001,1,1,1,593440,1,None,60,0,5,9,...,673,99,14,31,0,180,77,0,64,40


In [1210]:
P1001 = P1001['p2_2'].to_frame()
P1001.head(1)

,p2_2
CVE_MUN,
01001,593440


Exportar archivos

In [1211]:
# Metadatos
meta = meta           # Utiliza el archivo de metadatos que habías definido anteriormente
meta.at['Dataset base','Descripcion'] = '"P1001.xlsx" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015'
meta.at['Notas','Descripcion'] = 'p2_2: Cantidad de residuos sólidos recolectada en kilogramos.'
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,p2_2: Cantidad de residuos sólidos recolectada en kilogramos.
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P1001.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1213]:
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD'+'\\'+param+'.xlsx'
writer = pd.ExcelWriter(file)
P1001.to_excel(writer, sheet_name = param)
meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()

### P1006 - Número de municipios con aplicación de programas locales orientados a la GIRSU

In [1214]:
# Rutas de archivos
param = 'P1006'
rutadatos = tablas[param]
rutameta = tablas[param].replace('conjunto_de_datos', 'diccionario_de_datos')
rutameta = rutameta + '\\' + os.listdir(rutameta)[0]
print('{}\n{}'.format(rutadatos, rutameta))

D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1006\Prog_gest_int_RSU_cngmd2015_csv\conjunto_de_datos
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1006\Prog_gest_int_RSU_cngmd2015_csv\diccionario_de_datos\FD_Modulo_6_RSU_2015_secc_v.csv


¿Donde estan los datos?

El archivo secc_v_tr_cngmd15_m6.csv Contiene variables que caracterizan a los municipios de acuerdo a los programas orientados a la gestión integral de los residuos sólidos urbanos, durante el año 2014. En este archivo, la columna P13 Indica si se cuenta con algún programa orientado a la gestión integral de residuos sólidos urbanos (1 = Cuenta con Programas; 2 = No cuenta con programas).

El archivo secc_v_tr_cngmd15_m6_p13_1.csv Contiene la variable P13_1_1_2, que indica el tipo de programa orientado a la gestión integral de residuos sólidos urbanos.

In [1215]:
# Definición de rutas a archivos de datos
Paramfiles = {}
for file in os.listdir(rutadatos):
    Paramfiles[file.replace('.csv', '')] = rutadatos+'\\'+file

for file, path in Paramfiles.items():
    print('{}:\n{}\n'.format(file, path))

secc_v_tr_cngmd15_m6:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1006\Prog_gest_int_RSU_cngmd2015_csv\conjunto_de_datos\secc_v_tr_cngmd15_m6.csv

secc_v_tr_cngmd15_m6_p13_1:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1006\Prog_gest_int_RSU_cngmd2015_csv\conjunto_de_datos\secc_v_tr_cngmd15_m6_p13_1.csv



In [1216]:
os.listdir(tablas[param])[0]

'secc_v_tr_cngmd15_m6.csv'

In [1217]:
# Carga de datos
P1006f = tablas[param]+'\\'+ os.listdir(tablas[param])[0]
df = pd.read_csv(P1006f, dtype={'folio':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'folio':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1006 = df.where((pd.notnull(df)), None)

Exportar Archivos

In [1218]:
P1006 = P1006['p13'].to_frame()

In [1219]:
# Metadatos
meta = meta           # Utiliza el archivo de metadatos que habías definido anteriormente
meta.at['Dataset base','Descripcion'] = '"P1006.xlsx" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015'
meta.at['Notas','Descripcion'] = 'En la columna p13, ¿El municipio cuenta con Programas de Gestion de Residuos? 1: Si, 2: No'

In [1220]:
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,"En la columna p13, ¿El municipio cuenta con Programas de Gestion de Residuos? 1: Si, 2: No"
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P1006.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1222]:
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD'+'\\'+param+'.xlsx'
writer = pd.ExcelWriter(file)
P1006.to_excel(writer, sheet_name = param)
meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()

### P1009 - Número de municipios con estudios de generación de RSU

In [1325]:
# Rutas de archivos
param = 'P1009'
rutadatos = tablas[param]
rutameta = tablas[param].replace('conjunto_de_datos', 'diccionario_de_datos')
rutameta = rutameta + '\\' + os.listdir(rutameta)[0]
print('{}\n{}'.format(rutadatos, rutameta))

D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1009\Est_gen_comp_RSU_cngmd2015_csv\conjunto_de_datos
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1009\Est_gen_comp_RSU_cngmd2015_csv\diccionario_de_datos\FD_Modulo_6_RSU_2015_secc_iv.csv


¿Donde están los datos?

secc_iv_tr_cngmd15_m6 Contiene variables que caracterizan a los municipios de acuerdo a los estudios sobre la generación y composición de los residuos sólidos urbanos, durante el año 2014. 

La columna P12 Indica si se cuenta con algún estudio sobre la generación de residuos sólidos urbanos (1 = Si; 2 = No).

In [1326]:
# Definición de rutas a archivos de datos
Paramfiles = {}
for file in os.listdir(rutadatos):
    Paramfiles[file.replace('.csv', '')] = rutadatos+'\\'+file

for file, path in Paramfiles.items():
    print('{}:\n{}\n'.format(file, path))

secc_iv_tr_cngmd15_m6:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1009\Est_gen_comp_RSU_cngmd2015_csv\conjunto_de_datos\secc_iv_tr_cngmd15_m6.csv



In [1334]:
# Carga de datos
P1009f = tablas[param]+'\\'+ os.listdir(tablas[param])[0]
df = pd.read_csv(P1009f, dtype={'folio':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'folio':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1009 = df.where((pd.notnull(df)), None)
del(P1009['entidad'])
del(P1009['municipio'])

Exportar archivos

In [1328]:
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,s/n
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P0307.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1335]:
# Metadatos
meta = meta           # Utiliza el archivo de metadatos que habías definido anteriormente
meta.at['Dataset base','Descripcion'] = '"P1009.xlsx" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015'
meta.at['Notas','Descripcion'] = 'Para la columna P12, ¿El Municipio cuenta con estudios de generacion de residuos? 1: Si 2: No'
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,"Para la columna P12, ¿El Municipio cuenta con estudios de generacion de residuos? 1: Si 2: No"
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P1009.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1336]:
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD'+'\\'+param+'.xlsx'
writer = pd.ExcelWriter(file)
P1009.to_excel(writer, sheet_name = param)
meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()

### P1003 - Número de municipios con disponibilidad de servicios relacionados con los RSU

__¿Donde estan los datos?__

La informacion de este parametro se encuentra dividida entre diferentes carpetas.

In [1337]:
tablasP1003

{'1': 'D:\\PCCS\\00_RawData\\01_CSV\\cngmd\\2015\\P1003\\1\\Rec_RSU_cngmd2015_csv\\conjunto_de_datos',
 '2': 'D:\\PCCS\\00_RawData\\01_CSV\\cngmd\\2015\\P1003\\2\\Trat_RSU_cngmd2015_csv\\conjunto_de_datos',
 '3': 'D:\\PCCS\\00_RawData\\01_CSV\\cngmd\\2015\\P1003\\3\\Disp_final_RSU_cngmd2015_csv\\conjunto_de_datos'}

__La Carpeta 1 Contiene 2 archivos:__

secc_i_tr_cngmd15_m6.csv - Contiene variables que caracterizan a los municipios de acuerdo a la recolección de residuos sólidos urbanos, durante el año 2014. En este archivo, la variable P1 indica la disponibilidad del servicio de recolección (1: Si, 2:No)

secc_i_tr_cngmd15_m6_p6_3_2.csv - Contiene variables que caracterizan a los municipios de acuerdo al parque vehicular utilizado para la recolección y traslado de residuos sólidos urbanos, durante el año 2014. En este archivo, la variable P6_3_2_1_3 contiene el número de vehículos utilizados para la recolección de Residuos solidos urbanos. (Esta variable puede utilizarse para la construcción del parámetro 1005)

__La Carpeta 2 Contiene 1 archivo:__

secc_ii_tr_cngmd15_m6.csv - Contiene variables que caracterizan a los municipios de acuerdo al tratamiento de los residuos, durante el año 2014. En este archivo, la variable P10 Identifica si al menos una fracción de los residuos sólidos urbanos recolectados por el municipio o delegación es enviada a plantas de tratamiento (1: Si, 2:No)

__La Carpeta 3 contiene 1 archivo:__
secc_iii_tr_cngmd15_m6.csv - Contiene variables que caracterizan a los municipios de acuerdo a la disposición final de los residuos sólidos urbanos, durante el año 2014. En este archivo, la variable P11 Identifica el número de sitios de disposición final a los que son son remitidos los residuos que se recolectan en todo el municipio o delegación


In [1347]:
# Rutas de archivos
param = 'P1003'
rutasdatos = list(tablasP1003.values())
for ruta in rutasdatos:
    print(ruta)

D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\1\Rec_RSU_cngmd2015_csv\conjunto_de_datos
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\2\Trat_RSU_cngmd2015_csv\conjunto_de_datos
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\3\Disp_final_RSU_cngmd2015_csv\conjunto_de_datos


In [1348]:
# Definición de rutas a archivos de datos
Paramfiles = {}
for rutadatos in rutasdatos:
    for file in os.listdir(rutadatos):
        Paramfiles[file.replace('.csv', '')] = rutadatos+'\\'+file

for file, path in Paramfiles.items():
    print('{}:\n{}\n'.format(file, path))

secc_i_tr_cngmd15_m6:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\1\Rec_RSU_cngmd2015_csv\conjunto_de_datos\secc_i_tr_cngmd15_m6.csv

secc_i_tr_cngmd15_m6_p6_3_2:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\1\Rec_RSU_cngmd2015_csv\conjunto_de_datos\secc_i_tr_cngmd15_m6_p6_3_2.csv

secc_ii_tr_cngmd15_m6:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\2\Trat_RSU_cngmd2015_csv\conjunto_de_datos\secc_ii_tr_cngmd15_m6.csv

secc_iii_tr_cngmd15_m6:
D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\3\Disp_final_RSU_cngmd2015_csv\conjunto_de_datos\secc_iii_tr_cngmd15_m6.csv



In [1349]:
# Carga de datos
# Es necesario hacer 3 dataframes, uno por cada archivo, y después unir las columnas para cada parámetro.
P1003f1 = Paramfiles['secc_i_tr_cngmd15_m6']
df = pd.read_csv(P1003f1, dtype={'folio':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'folio':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1003f1 = df.where((pd.notnull(df)), None)

P1003f2 = Paramfiles['secc_ii_tr_cngmd15_m6']
df = pd.read_csv(P1003f2, dtype={'folio':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'folio':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1003f2 = df.where((pd.notnull(df)), None)

# El Parametro en realidad no utiliza el numero de sitios de disposicion de residuos. 
# Y no está documentado el significado de NS en la columna P11 lo que dificulta la lectura de los datos
''' 
P1003f3 = Paramfiles['secc_iii_tr_cngmd15_m6']
df = pd.read_csv(P1003f3, dtype={'folio':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'folio':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1003f3 = df.where((pd.notnull(df)), None)
'''

" \nP1003f3 = Paramfiles['secc_iii_tr_cngmd15_m6']\ndf = pd.read_csv(P1003f3, dtype={'folio':'str'}, encoding = 'mbcs')\ndf = df.rename(columns = {'folio':'CVE_MUN'})\ndf.set_index('CVE_MUN', inplace = True)\nP1003f3 = df.where((pd.notnull(df)), None)\n"

In [1351]:
# Aislar datos de interés
P1003 = P1003f1['p1'].to_frame()
P1003['p10'] = P1003f2['p10']
# P1003['p11'] = P1003f3['p11']   #p11 se excluye del analisis por los motivos descritos antes

P1003.head(1)

,p1,p10
CVE_MUN,,
01001,1,2


Exportar archivos

In [1352]:
# Metadatos
meta = meta           # Utiliza el archivo de metadatos que habías definido anteriormente
meta.at['Dataset base','Descripcion'] = '"P1003.xlsx" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015'
meta.at['Notas','Descripcion'] = 'para p1: ¿Dispone de servicio de recoleccion? (1: Si 2: No)\npara p10: ¿Al menos una fracción de los RSU es enviada a plantas de tratamiento? (1: Si 2: No)\npara p11: ¿A cuantos sitios de disposición final son remitidos los residuos?'
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,para p1: ¿Dispone de servicio de recoleccion? (1: Si 2: No)\npara p10: ¿Al menos una fracción de los RSU es enviada a pla...
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P1003.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1353]:
param

'P1003'

In [1355]:
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD'+'\\'+param+'.xlsx'
writer = pd.ExcelWriter(file)
P1003.to_excel(writer, sheet_name = param)
meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()

### P1005 - Número de vehículos utilizados para la recolección de residuos sólidos urbanos

¿Donde están los datos?

__La Carpeta 1 de P1003 (Procesada previamente) contiene 2 archivos:__

secc_i_tr_cngmd15_m6.csv, y

secc_i_tr_cngmd15_m6_p6_3_2.csv - Contiene variables que caracterizan a los municipios de acuerdo al parque vehicular utilizado para la recolección y traslado de residuos sólidos urbanos, durante el año 2014. En este archivo, la variable P6_3_2_1_3 contiene el número de vehículos utilizados para la recolección de Residuos solidos urbanos. (Esta variable puede utilizarse para la construcción del parámetro 1005)

In [1358]:
# Carga de datos
P1005f = Paramfiles['secc_i_tr_cngmd15_m6_p6_3_2']
df = pd.read_csv(P1005f, dtype={'FOLIO':'str'}, encoding = 'mbcs')
df = df.rename(columns = {'FOLIO':'CVE_MUN'})
df.set_index('CVE_MUN', inplace = True)
P1005f = df.where((pd.notnull(df)), None)

In [1357]:
P1005f.head(1)

,ID_GR085331290,P6_3_2_1_1,P6_3_2_1_3
ubic_geo,,,
01001,488,9,12


Exportar archivos

In [1359]:
P1005 = P1005f['P6_3_2_1_3'].to_frame()
P1005.head(3)

,P6_3_2_1_3
CVE_MUN,
01001,12
01001,15
01001,11


In [1360]:
# Metadatos
meta = meta           # Utiliza el archivo de metadatos que habías definido anteriormente
meta.at['Dataset base','Descripcion'] = '"P1005.xlsx" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015'
meta.at['Notas','Descripcion'] = 'P6_3_2_1_3: Numero de vehiculos utilizados para la recolección de Residuos Solidos Urbanos'
meta

,Descripcion
Metadato,
Nombre del Dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Descripcion del dataset,Censo Nacional de Gobiernos Municipales y Delegacionales 2015
Disponibilidad Temporal,2015
Periodo de actualizacion,Bienal
Nivel de Desagregacion,Municipal
Notas,P6_3_2_1_3: Numero de vehiculos utilizados para la recolección de Residuos Solidos Urbanos
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/
Dataset base,"""P1005.xlsx"" disponible en \nhttps://github.com/INECC-PCCS/01_Dmine/tree/master/Datasets/CNGMD/2015"


In [1361]:
param = 'P1005'
file = r'D:\PCCS\01_Dmine\Datasets\CNGMD'+'\\'+param+'.xlsx'
writer = pd.ExcelWriter(file)
P1005.to_excel(writer, sheet_name = param)
meta.to_excel(writer, sheet_name ='METADATOS')
writer.save()